In [23]:
import pandas as pd
import numpy as np
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [2]:
df = pd.read_csv("data/train.tsv", sep='\t')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [3]:
df["Sentiment"].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [4]:
print("Mean of word count in test set",np.mean(df['Phrase'].apply(lambda x: len(x.split()))))
print("Max of word count in test set",np.max(df['Phrase'].apply(lambda x: len(x.split()))))
print("Min of word count in test set",np.min(df['Phrase'].apply(lambda x: len(x.split()))))

Mean of word count in test set 7.203364090734333
Max of word count in test set 52
Min of word count in test set 0


In [9]:
results = set()
df['Phrase'].str.lower().str.split().apply(results.update)
print("Unique  words in phrases", len(results))

Unique  words in phrases 16531


In [11]:
x = df["Phrase"]
y = df["Sentiment"]

trainx, testx, trainy, testy = train_test_split(x,y, test_size=0.33, random_state=42)

In [15]:
# integer encode the documents
vocab_size = 17000
encoded_docs = [one_hot(d, vocab_size) for d in x]
# print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 55 
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, y, epochs=50, verbose=2)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, y, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 55, 8)             136000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 440)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 441       
Total params: 136,441
Trainable params: 136,441
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
 - 21s - loss: -1.6583e+01 - acc: 0.1747
Epoch 2/50
 - 20s - loss: -1.6956e+01 - acc: 0.1748
Epoch 3/50
 - 21s - loss: -1.6958e+01 - acc: 0.1748
Epoch 4/50
 - 21s - loss: -1.6963e+01 - acc: 0.1749
Epoch 5/50
 - 24s - loss: -1.6971e+01 - acc: 0.1751
Epoch 6/50
 - 26s - loss: -1.6983e+01 - acc: 0.1754
Epoch 7/50
 - 23s - loss: -1.7001e+01 - acc: 0.1757
Epoch 8/50
 - 26s - loss: -1.70

KeyboardInterrupt: 

In [22]:
model=Sequential()
model.add(Embedding(17000,250,mask_zero=True))
model.add(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 250)         4250000   
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 128)         194048    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
Total params: 4,493,781
Trainable params: 4,493,781
Non-trainable params: 0
_________________________________________________________________


In [24]:
target = to_categorical(y)

In [ ]:
%%time
history=model.fit(padded_docs, target, epochs=5, batch_size=50, verbose=1)

Epoch 1/5
 35550/156060 [=====>........................] - ETA: 16:42 - loss: 1.1765 - acc: 0.5434